In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree

## Helper Functions

In [4]:
def numeralize_categories(df:pd.DataFrame) -> list:
  '''
  receives a dataframe identifies the columns of type category and returns a list of tupples, each tupple 
  contains the name of the column and a dictionary with each unique value as a key and the number associated 
  with it.
  '''
  cats = []
  for column in df.columns:
    if df.dtypes[column] == 'category':
       cats.append((column, {j:i for i,j in enumerate(df[column].unique(), 0)}))
  return cats

In [5]:
def numeralize_column(col: pd.Series) -> tuple:
  '''
  receives a series evaluates if is an object type and returns a tupple with the name of the series and
  a dictionary with the unique values asociated
  '''
  if col.dtype == 'object':
    return col.name, {j:i for i,j in enumerate(col.unique(), 0)}

In [6]:
def numeralize_columns(categories_map: list, df: pd.DataFrame):
  '''
  receives a list of tupples (category name, dictionary{unique values}) and a dataframe. The dataframe columns
  contained in the categories_map list are replaced with the values of the dictionary 
  '''
  for cat, items in categories_map:
    df[cat].replace(items, inplace=True)
  

In [7]:
def scatter_plot(x: pd.Series, y: pd.Series, color: str):
  data = [go.Scatter(x=x,
                     y=y,
                     mode='markers',
                     marker= dict(size=12,
                                  color=color,
                                  symbol='circle-dot',
                                  line={'width': 2}))]
  layout = go.Layout(title='{} vs {} correlation'.format(x.name, y.name),
                    xaxis={'title': x.name},
                    yaxis={'title': y.name},
                    hovermode='closest',
                    template='plotly_white')
  fig = go.Figure(data=data, layout=layout)

  fig.show()
  # pyo.plot(fig, filename='{}_{}_correlation.html'.format(x.name, y.name))

In [8]:
def get_categories_bar_plot(df: pd.DataFrame, categories: list, value_name: str):
  data = [go.Bar(x=df.index, y=df[cat], name=cat) for cat in categories]
  layout = go.Layout(title='{} by {}'.format(value_name, df.columns.name))

  fig = go.Figure(data=data, layout=layout)
  fig.show()

In [9]:
def get_scatter_with_marker(df: pd.DataFrame, metadata: dict):
  '''
    metadata {
      x: x axis
      y: y axis
      text: each point label
      mode: chart mode
      marker: markers parameters
    }
  '''
  data = [go.Scatter(x=df[metadata['x']], 
                     y=df[metadata['y']],
                     text = df[metadata['text']],
                     mode=metadata['mode'],
                     marker=dict(size=df[metadata['marker']]))]
  layout = go.Layout(title='{} by {} and {}'.format(metadata['x'], metadata['y'], metadata['marker']))
  return go.Figure(data=data, layout=layout)
  

In [10]:
"""
Separate X and Y
"""

def get_variables(df: pd.DataFrame, dependent: str) -> list:
  '''
  Receives the desired dataframe to separate and the column name to be identified as dependent variable
  returns a list containing a DataFrame for independent variables and a Series with the dependent variable
  '''
  if dependent in df.columns:
    x = df.copy(deep=True)
    y = x[dependent]
    x.drop('stroke', axis='columns', inplace=True)
    return [x, y]
  else:
    return None

In [11]:
file = 'data/healthcare-dataset-stroke-data.csv'

In [12]:
raw_data = pd.read_csv(file)

We don't need the `id` column so I am dropping it

In [13]:
raw_data.drop(['id'], axis=1, inplace=True)

#### Missing Data

In [14]:
raw_data[raw_data.isna().values]

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
8,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
13,Male,78.0,0,1,Yes,Private,Urban,219.84,NaN,Unknown,1
19,Male,57.0,0,1,No,Govt_job,Urban,217.08,NaN,Unknown,1
27,Male,58.0,0,0,Yes,Private,Rural,189.84,NaN,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...
5039,Male,41.0,0,0,No,Private,Rural,70.15,NaN,formerly smoked,0
5048,Male,40.0,0,0,Yes,Private,Urban,191.15,NaN,smokes,0
5093,Female,45.0,1,0,Yes,Govt_job,Rural,95.02,NaN,smokes,0
5099,Male,40.0,0,0,Yes,Private,Rural,83.94,NaN,smokes,0


a first look shows that we have nan values in `bmi` column, however sometimes missing values are present in other forms, like the `unknown` in the `smoke` column

In [296]:
for column in raw_data.columns:
  print(raw_data[column].unique())

['Male' 'Female' 'Other']
[6.70e+01 6.10e+01 8.00e+01 4.90e+01 7.90e+01 8.10e+01 7.40e+01 6.90e+01
 5.90e+01 7.80e+01 5.40e+01 5.00e+01 6.40e+01 7.50e+01 6.00e+01 5.70e+01
 7.10e+01 5.20e+01 8.20e+01 6.50e+01 5.80e+01 4.20e+01 4.80e+01 7.20e+01
 6.30e+01 7.60e+01 3.90e+01 7.70e+01 7.30e+01 5.60e+01 4.50e+01 7.00e+01
 6.60e+01 5.10e+01 4.30e+01 6.80e+01 4.70e+01 5.30e+01 3.80e+01 5.50e+01
 1.32e+00 4.60e+01 3.20e+01 1.40e+01 3.00e+00 8.00e+00 3.70e+01 4.00e+01
 3.50e+01 2.00e+01 4.40e+01 2.50e+01 2.70e+01 2.30e+01 1.70e+01 1.30e+01
 4.00e+00 1.60e+01 2.20e+01 3.00e+01 2.90e+01 1.10e+01 2.10e+01 1.80e+01
 3.30e+01 2.40e+01 3.40e+01 3.60e+01 6.40e-01 4.10e+01 8.80e-01 5.00e+00
 2.60e+01 3.10e+01 7.00e+00 1.20e+01 6.20e+01 2.00e+00 9.00e+00 1.50e+01
 2.80e+01 1.00e+01 1.80e+00 3.20e-01 1.08e+00 1.90e+01 6.00e+00 1.16e+00
 1.00e+00 1.40e+00 1.72e+00 2.40e-01 1.64e+00 1.56e+00 7.20e-01 1.88e+00
 1.24e+00 8.00e-01 4.00e-01 8.00e-02 1.48e+00 5.60e-01 4.80e-01 1.60e-01]
[0 1]
[1 0]
['Yes' 'No']

In [15]:
for i, row in raw_data.iterrows():
  if pd.isna(row['bmi']):
    raw_data.iat[i, raw_data.columns.get_loc('bmi')] = raw_data.loc[raw_data['age'] == row['age']].bmi.median()
    


In [17]:
raw_data.to_csv('stroke_data.csv')

In [16]:
raw_data[raw_data.isna().values]

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke


In [13]:
raw_data.loc[:, raw_data.dtypes == 'object'] = raw_data.select_dtypes(['object']).apply(lambda x: x.astype('category'))

In [299]:
raw_data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.60,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,29.45,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.50,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.40,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.00,never smoked,1


In [200]:
import plotly.offline as pyo
import plotly.graph_objs as go

In [228]:
scatter_plot(raw_data.age, raw_data.bmi, '#dab894')

In [229]:
scatter_plot(raw_data.avg_glucose_level, raw_data.bmi, '#dab894')

In [244]:
stroke_by_smoking = pd.pivot_table(raw_data, values='stroke', index=['smoking_status'], columns=['gender'], aggfunc=np.sum)

In [245]:
stroke_by_smoking

gender,Female,Male,Other
smoking_status,,,
Unknown,25.0,22.0,NaN
formerly smoked,34.0,36.0,0.0
never smoked,63.0,27.0,NaN
smokes,19.0,23.0,NaN


In [232]:
stroke_by_smoking['Total'] = stroke_by_smoking['Female'] + stroke_by_smoking['Male'] + stroke_by_smoking['Other']

In [233]:
stroke_by_smoking

gender,Female,Male,Other,Total
smoking_status,,,,
Unknown,25.0,22.0,NaN,NaN
formerly smoked,34.0,36.0,0.0,70.0
never smoked,63.0,27.0,NaN,NaN
smokes,19.0,23.0,NaN,NaN


In [262]:
get_categories_bar_plot(stroke_by_smoking, stroke_by_smoking.columns, 'strokes')

In [238]:

trace1 = go.Bar(x=stroke_by_smoking.index, y=stroke_by_smoking['Female'], name='Female')
trace2 = go.Bar(x=stroke_by_smoking.index, y=stroke_by_smoking['Male'], name='Male')
trace3 = go.Bar(x=stroke_by_smoking.index, y=stroke_by_smoking['Other'], name='Other')
data = [trace1,trace2,trace3]
layout = go.Layout(title='Strokes by Smoking Status')
# layout = go.Layout(title='Strokes by Smoking Status', barmode='stack') for stack the bars by category
fig = go.Figure(data=data, layout=layout)
# pyo.plot(fig)
fig.show()

In [23]:
df_stroke_never_smoked_F = raw_data.loc[(raw_data['stroke'] == 1) & (raw_data.gender == 'Female' ) & (raw_data['smoking_status'] == 'never smoked')]

In [24]:
df_stroke_never_smoked_F

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,30.19,never smoked,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.00,never smoked,1
7,Female,69.0,0,0,No,Private,Urban,94.39,22.80,never smoked,1
10,Female,81.0,1,0,Yes,Private,Rural,80.43,29.70,never smoked,1
14,Female,79.0,0,1,Yes,Private,Urban,214.09,28.20,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
226,Female,81.0,0,0,Yes,Private,Rural,184.40,27.50,never smoked,1
229,Female,80.0,1,0,No,Private,Urban,66.03,35.40,never smoked,1
230,Female,81.0,0,0,Yes,Self-employed,Rural,81.95,16.90,never smoked,1
232,Female,73.0,0,0,Yes,Self-employed,Urban,70.94,34.40,never smoked,1


In [285]:
params = {'x': 'avg_glucose_level',
          'y': 'age',
          'text': 'bmi',
          'mode': 'markers',
          'marker': 'bmi'
          }

get_scatter_with_marker(df_stroke_never_smoked_F, params).show()

In [27]:
df_female_stroke = raw_data.loc[(raw_data['stroke'] == 1) & (raw_data['gender'] == 'Female')]

In [28]:
df_female_stroke

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,30.190,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000,never smoked,1
7,Female,69.0,0,0,No,Private,Urban,94.39,22.800,never smoked,1
8,Female,59.0,0,0,Yes,Private,Rural,76.15,31.192,Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...
238,Female,51.0,1,0,Yes,Private,Urban,88.20,28.400,never smoked,1
243,Female,68.0,1,1,Yes,Private,Urban,247.51,40.500,formerly smoked,1
245,Female,14.0,0,0,No,children,Rural,57.93,30.900,Unknown,1
246,Female,75.0,0,0,Yes,Self-employed,Rural,78.80,29.300,formerly smoked,1


In [286]:
get_scatter_with_marker(df_female_stroke, params).show()

In [31]:
df_male_stroke = raw_data.loc[(raw_data['stroke'] == 1) & (raw_data['gender'] == 'Male')]

In [287]:
get_scatter_with_marker(df_male_stroke, params).show()

## Sampling

-  Split dependent and independent variables
-  Import SMOTE library

In [33]:
'''
1
'''
x = raw_data.drop('stroke', axis='columns')
y = raw_data['stroke']

In [34]:
y.value_counts()

0    4861
1     249
Name: stroke, dtype: int64

In [41]:
raw_data[raw_data.isna().values]

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke


In [42]:
raw_data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.60,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,30.19,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.50,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.40,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.00,never smoked,1


In [43]:
# Change objects dtypes to categrocial and confirm with info()
raw_data.loc[:,raw_data.dtypes == 'object'] = raw_data.select_dtypes(['object']).apply(lambda x: x.astype('category'))
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   gender             5110 non-null   category
 1   age                5110 non-null   float64 
 2   hypertension       5110 non-null   int64   
 3   heart_disease      5110 non-null   int64   
 4   ever_married       5110 non-null   category
 5   work_type          5110 non-null   category
 6   Residence_type     5110 non-null   category
 7   avg_glucose_level  5110 non-null   float64 
 8   bmi                5110 non-null   float64 
 9   smoking_status     5110 non-null   category
 10  stroke             5110 non-null   int64   
dtypes: category(5), float64(3), int64(3)
memory usage: 265.4 KB


In [153]:
raw_data.gender.unique()

[1, 0, 2]
Categories (3, int64): [0, 1, 2]

In [45]:
raw_data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.60,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,30.19,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.50,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.40,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.00,never smoked,1


### Numerical params for cathegorical data
- `gender` - [Male: 1, Female: 0, other: 2]
- `ever_maried` - [yes: 1, no: 0]
- `work` - [Private: 1, Self-employed: 2, Gov_jov: 3, children: 4, Never_worked: 5]
- `Residence_type` - [Rural: 0, Urban: 1]
- `smoking_status` - [Unknown: 0, formely smoked: 1, never smoked: 2, smokes: 3]

In [56]:
df = raw_data

In [70]:
df.describe

<bound method NDFrame.describe of      gender   age  hypertension  heart_disease ever_married work_type  \
0         1  67.0             0              1          Yes         0   
1         0  61.0             0              0          Yes         1   
2         1  80.0             0              1          Yes         0   
3         0  49.0             0              0          Yes         0   
4         0  79.0             1              0          Yes         1   
...     ...   ...           ...            ...          ...       ...   
5105      0  80.0             1              0          Yes         0   
5106      0  81.0             0              0          Yes         1   
5107      0  35.0             0              0          Yes         1   
5108      1  51.0             0              0          Yes         0   
5109      0  44.0             0              0          Yes         2   

     Residence_type  avg_glucose_level        bmi smoking_status  stroke  
0             

In [290]:
categories = []
for column in raw_data.columns:
  if raw_data[column].dtype == 'object':
    categories.append(numeralize_column(raw_data[column]))

In [291]:
categories

[('gender', {'Male': 0, 'Female': 1, 'Other': 2}),
 ('ever_married', {'Yes': 0, 'No': 1}),
 ('work_type',
  {'Private': 0,
   'Self-employed': 1,
   'Govt_job': 2,
   'children': 3,
   'Never_worked': 4}),
 ('Residence_type', {'Urban': 0, 'Rural': 1}),
 ('smoking_status',
  {'formerly smoked': 0, 'never smoked': 1, 'smokes': 2, 'Unknown': 3})]

In [300]:
df = raw_data
numeralize_columns(categories, df)

In [301]:
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,67.0,0,1,0,0,0,228.69,36.60,0,1
1,1,61.0,0,0,0,1,1,202.21,29.45,1,1
2,0,80.0,0,1,0,0,1,105.92,32.50,1,1
3,1,49.0,0,0,0,0,0,171.23,34.40,2,1
4,1,79.0,1,0,0,1,1,174.12,24.00,1,1


In [306]:
df.loc[:,df.dtypes == 'object'] = df.select_dtypes(['object']).apply(lambda x: x.astype('category'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   int64  
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   int64  
 5   work_type          5110 non-null   int64  
 6   Residence_type     5110 non-null   int64  
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                5110 non-null   float64
 9   smoking_status     5110 non-null   int64  
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(8)
memory usage: 439.3 KB


In [307]:
from imblearn.over_sampling import SMOTE

In [308]:
y = df.stroke
x = df
x.drop('stroke', axis='columns', inplace=True)
y.value_counts()

0    4861
1     249
Name: stroke, dtype: int64

In [309]:
smote = SMOTE(random_state=12)
x_sm, y_sm = smote.fit_resample(x, y)
y_sm.value_counts()

1    4861
0    4861
Name: stroke, dtype: int64

In [140]:
df_oversampled = x_sm
df_oversampled['stroke'] = y_sm

In [305]:
y_sm.head()

0    1
1    1
2    1
3    1
4    1
Name: stroke, dtype: int64

In [310]:
df_oversampled[~df_oversampled.isna()]

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.000000,0,1,1,0,1,228.690000,36.600000,1,1
1,0,61.000000,0,0,1,1,0,202.210000,30.190000,2,1
2,1,80.000000,0,1,1,0,0,105.920000,32.500000,2,1
3,0,49.000000,0,0,1,0,1,171.230000,34.400000,3,1
4,0,79.000000,1,0,1,1,0,174.120000,24.000000,2,1
...,...,...,...,...,...,...,...,...,...,...,...
9717,1,60.811876,0,0,1,0,NaN,111.737572,27.742092,NaN,1
9718,1,72.799565,0,0,1,0,NaN,139.410676,30.343137,1,1
9719,0,77.486355,0,0,1,0,NaN,228.381636,31.253100,NaN,1
9720,0,57.723217,0,0,1,0,NaN,97.190938,26.400000,NaN,1


In [79]:
df_oversampled.gender.replace({
  1: 'Male',
  0: 'Female',
  2: 'Other'
}, inplace=True)

df_oversampled.ever_married.replace({
  1: 'Yes',
  0: 'No'
}, inplace=True)

df_oversampled.work_type.replace({
  0: 'Private',
  1: 'Self-employed',
  2: 'Govt_job',
  3: 'children',
  4: 'Never_worked'
}, inplace=True)

df_oversampled.Residence_type.replace({
  0: 'Rural',
  1: 'Urban'
}, inplace=True)

df_oversampled.smoking_status.replace({
  0: 'Unknown',
  1: 'formerly smoked',
  2: 'never smoked',
  3: 'smokes'
}, inplace=True)

In [96]:
df_oversampled.dtypes['gender']

CategoricalDtype(categories=['Female', 'Male', 'Other'], ordered=False)

In [110]:


print(numeralize_categories(df_oversampled))

# for tup in cats:
#   for item in tup:
#     print(item)

[('gender', {'Male': 0, 'Female': 1, 'Other': 2, nan: 3}), ('ever_married', {'Yes': 0, 'No': 1, nan: 2}), ('work_type', {'Private': 0, 'Self-employed': 1, 'Govt_job': 2, 'children': 3, 'Never_worked': 4, nan: 5}), ('Residence_type', {'Urban': 0, 'Rural': 1, nan: 2}), ('smoking_status', {'formerly smoked': 0, 'never smoked': 1, 'smokes': 2, 'Unknown': 3, nan: 4})]


In [80]:
over_stroke_smoker = pd.pivot_table(df_oversampled, values='stroke', index=['smoking_status'], columns=['gender'], aggfunc=np.sum)

In [81]:
over_stroke_smoker['Total'] = over_stroke_smoker['Female'] + over_stroke_smoker['Male'] + over_stroke_smoker['Other']

In [82]:
over_stroke_smoker

gender,Female,Male,Other,Total
smoking_status,,,,
Unknown,71,93,0,164
formerly smoked,96,170,0,266
never smoked,354,110,0,464
smokes,34,66,0,100


In [83]:
trace1 = go.Bar(x=over_stroke_smoker.index, y=over_stroke_smoker['Female'], name='Female')
trace2 = go.Bar(x=over_stroke_smoker.index, y=over_stroke_smoker['Male'], name='Male')
trace3 = go.Bar(x=over_stroke_smoker.index, y=over_stroke_smoker['Other'], name='Other')
data = [trace1,trace2,trace3]
layout = go.Layout(title='Strokes by Smoking Status')
# layout = go.Layout(title='Strokes by Smoking Status', barmode='stack') for stack the bars by category
fig = go.Figure(data=data, layout=layout)
# pyo.plot(fig)
fig.show()

In [379]:
x, y = get_variables(raw_data, 'stroke')

In [381]:
x_train, y_train, x_test, y_test = train_test_split(x, y, test_size=0.3, train_size=0.7)

In [ ]:
def tree_classifier(depts: int, x_train, y_train, x_test, y_test):
  dt_classifier = tree.DecisionTreeClassifier(max_depth=depts)
  return dt_classifier.fit(x_train, y_train)

In [351]:
type(y)

pandas.core.series.Series

In [321]:
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,0,67.0,0,1,0,0,0,228.69,36.60,0
1,1,61.0,0,0,0,1,1,202.21,29.45,1
2,0,80.0,0,1,0,0,1,105.92,32.50,1
3,1,49.0,0,0,0,0,0,171.23,34.40,2
4,1,79.0,1,0,0,1,1,174.12,24.00,1
